In [1]:
import findspark
import os
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
# SPARK_HOME='/home/qiany/.conda/envs/py37'
# os.environ['SPARK_HOME'] = '/home/qiany/.conda/envs/py37'
findspark.init(SPARK_HOME)

In [2]:
import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
#from hdfs import InsecureClient
import numpy as np
#from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import create_map
from pyspark.sql.functions import array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
#import igraph
#from igraph import Graph
import geofeather
from pyspark.storagelevel import StorageLevel

In [3]:
tin_file = input("Here is a programe to compute the Forman gradient, please input the absolute or relative path to your TIN file:")
print("\n********************")

# get the directory to the TIN file
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or SigSpatial(1):") or "1"
if directory_type == '0':
    directory = 'hdfs_data'
else:
    directory = 'SigSpatial_data'
    
# get the basename to the TIN file
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

# get the filename of the TIN file
tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

# get the type of TIN file: off, tri, etc
tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)
print("\n********************")

Here is a programe to compute the Forman gradient, please input the absolute or relative path to your TIN file: SigSpatial_data/Canyon_Lake_Gorge_TX.off



********************
tin_directory:  SigSpatial_data


Is the data stored in hdfs(0) or SigSpatial(1): 1


tin_basename:  Canyon_Lake_Gorge_TX.off
tin_filename:  Canyon_Lake_Gorge_TX
tin_extension:  .off

********************


In [4]:
# filtra is the order of each vertex, the order is obtained by ranking elevation values of vertices
filtra = input("Do you have filtration data?") or "yes"
# filtra = 'yes'

if filtra.lower() == 'no':    
    Basic_Data = input("Do you have basic pts and tri data?")
    
Num_executor = input("spark.executor.instances:") or "64"
Num_core_per_executor = input("spark.executor.cores:") or "5"
Memory_executor = input("spark.executor.memory? Please end with 'g':") or "64g"
MemoryOverhead_executor = input("spark.executor.memoryOverhead? Please end with 'g':") or "8g"

# Num_core_per_driver = Num_core_per_executor
# Memory_driver = Memory_executor
# MemoryOverhead_driver = MemoryOverhead_executor

Num_core_per_driver = '5'
Memory_driver = '12g'
MemoryOverhead_driver = '4g'

Num_shuffle_partitions = input("spark.sql.shuffle.partitions:") or "200"

Do you have filtration data? yes
spark.executor.instances: 64
spark.executor.cores: 5
spark.executor.memory? Please end with 'g': 64g
spark.executor.memoryOverhead? Please end with 'g': 8g
spark.sql.shuffle.partitions: 200


In [5]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

In [6]:
os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [7]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

# set the Spark app name
spark_app_name = "Compute_2-manifolds_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '200') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.getOrCreate()

spark_app_name: Compute_2-manifolds_Canyon_Lake_Gorge_TX_07112024_2030


In [8]:
import sys
sys.path.append("/local/data/yuehui/pyspark/FormanGradient/code")

In [9]:
import graphframes
from graphframes import GraphFrame
from graphframes import *
from graphframes.lib import Pregel

In [10]:
# read_ver_order() is a function used to read vertices from a csv file
def read_ver_order(filtra, directory, tin_filename):
    '''
    this function has three input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a TIN file
    tin_filename: a string denoting the file name of a TIN without extension, e.g., Canyon_Lake_Gorge_TX
    '''
    if filtra.lower() == 'yes': # 'yes' means we have already computed the filtra value for each vertex
        schema_ver_origin = StructType([ \
            StructField("x",FloatType(),True), \
            StructField("y",FloatType(),True), \
            StructField("ele",FloatType(),True), \
            StructField("self_index",IntegerType(),True), \
            StructField("self_order",IntegerType(),True) \
          ])
        
        hdfs_tin_pts_origin = directory + "/" + tin_filename + '_filtra_pts_origin.csv'
    
        df_ver_order = spark.read.format("csv") \
              .option("header", False) \
              .schema(schema_ver_origin)\
              .load(hdfs_tin_pts_origin)
        print("Number of partitions for df_ver_order:", df_ver_order.rdd.getNumPartitions())
        
    if filtra.lower() == 'no': # 'no' means we need to rank vertices to get their filtra values
        schema_ver_origin = StructType([ \
            StructField("x",FloatType(),True), \
            StructField("y",FloatType(),True), \
            StructField("ele",FloatType(),True), \
            StructField("self_index",IntegerType(),True) \
          ])
        
        hdfs_tin_pts_origin = directory + "/" + tin_filename + '_pts_origin.csv'
    
        df_ver_origin = spark.read.format("csv") \
              .option("header", False) \
              .schema(schema_ver_origin)\
              .load(hdfs_tin_pts_origin)
        
        print("Number of partitions for df_ver_origin:", df_ver_origin.rdd.getNumPartitions())
        
        # define a window for the ordering
        # row_number() function along with partitionBy() of other column populates the row number by group
        # Since we want to order the whole DataFrame, so we don't need the partitionBy() function
        w = Window().orderBy(col('ele').asc())
        df_ver_order = df_ver_origin.withColumn("self_order", F.row_number().over(w) -1) # let the row number start from 0
        print("Number of partitions for df_ver_order:", df_ver_order.rdd.getNumPartitions())
        
    return df_ver_order

In [11]:
# read vertices
df_ver_order = read_ver_order(filtra, directory, tin_filename)
df_ver_order.printSchema()

Number of partitions for df_ver_order: 200
root
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- ele: float (nullable = true)
 |-- self_index: integer (nullable = true)
 |-- self_order: integer (nullable = true)



In [12]:
# read_tri_order() is a function used to read triangles from a csv file
def read_tri_order(filtra, directory, tin_filename):
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a TIN file
    tin_filename: a string denoting the file name of a tin without extension, e.g., 827_monviso
    '''
    if filtra.lower() == 'yes':
        hdfs_tin_tri_origin = directory + "/" + tin_filename + '_filtra_tri_origin.csv'
    else: # filtra.lower() == 'no'
        hdfs_tin_tri_origin = directory + "/" + tin_filename + '_tri_origin.csv'
        
    schema_tri_origin = StructType([ \
        StructField("v1",IntegerType(),True), \
        StructField("v2",IntegerType(),True), \
        StructField("v3",IntegerType(),True), \
        StructField("tri_order",IntegerType(),True) \
      ])

    df_tri_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tri_origin)\
          .load(hdfs_tin_tri_origin)
    print("Number of partitions for df_tri_origin:", df_tri_origin.rdd.getNumPartitions())
        
    return df_tri_origin

In [13]:
# read triangles
df_tri_origin = read_tri_order(filtra, directory, tin_filename)
df_tri_origin.printSchema()

Number of partitions for df_tri_origin: 200
root
 |-- v1: integer (nullable = true)
 |-- v2: integer (nullable = true)
 |-- v3: integer (nullable = true)
 |-- tri_order: integer (nullable = true)



In [14]:
# replace_ver() replaces the original index of each vertex with filtra value in df_tri_origin
def replace_ver(df_ver_order, df_tri_origin):
    '''
    df_ver_order: a DataFrame storing sorted vertices with filtra values
    df_tri_origin: a DataFrame storing triangles from a TIN
    '''
    df_tri_order_v1 = df_tri_origin.join(df_ver_order, df_tri_origin.v1 == df_ver_order.self_index, "inner")
    df_tri_order_v1 = df_tri_order_v1.select(col("tri_order"), col("v1"), col("v2"), col("v3"),col("self_order").alias("r1"), col("ele").alias("r1_ele"))

    df_tri_order_v2 = df_tri_order_v1.join(df_ver_order, df_tri_order_v1.v2 == df_ver_order.self_index, "inner")
    df_tri_order_v2 = df_tri_order_v2.select(col("tri_order"), col("v1"), col("v2"), col("v3"), col("r1"), col("self_order").alias("r2"), col("r1_ele"), col("ele").alias("r2_ele"))

    df_tri_order_v3 = df_tri_order_v2.join(df_ver_order, df_tri_order_v2.v3 == df_ver_order.self_index, "inner")
    df_tri_order_v3 = df_tri_order_v3.select(col("tri_order"), col("v1"), col("v2"), col("v3"), col("r1"), col("r2"), col("self_order").alias("r3"), col("r1_ele"), col("r2_ele"), col("ele").alias("r3_ele"))

    df_tri_order = df_tri_order_v3.select(col("tri_order"), col("r1"), col("r2"), col("r3"), col("r1_ele"), col("r2_ele"), col("r3_ele"))

    return df_tri_order

In [15]:
# replaces original vertex index with filtra values
df_tri_order = replace_ver(df_ver_order, df_tri_origin)
df_tri_order.printSchema()

root
 |-- tri_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)



In [16]:
# sort the extreme vertices of a triangle in ascending order, e.g., (2,5,3) -> (2,3,5)

'''
def get_tri_array(r1, r2, r3):
# get_multi_pt_index is used to obtain the adjacent vertexes index, including the vertex itself
# pt_list1: partial adjacent vertex indexes of join result 1
# pt_list2: partial adjacent vertex indexes of join result 2
    tri = [r1, r2, r3]    
    tri.sort(reverse=True)
    
    return tri

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_tri_array_udf = udf(get_tri_array, ArrayType(IntegerType()))
df_tri_order = df_tri_order.withColumn("tri", get_tri_array_udf(df_tri_order.r1, df_tri_order.r2, df_tri_order.r3))
'''

df_tri_order = df_tri_order.withColumn("tri_origin", F.array("r1", "r2", "r3")).withColumn("tri_ele_origin", F.array("r1_ele", "r2_ele", "r3_ele"))
df_tri_order = df_tri_order.withColumn("tri", sort_array("tri_origin", False)).drop("tri_origin") # sort_array("tri", False), False means descending order
df_tri_order = df_tri_order.withColumn("tri_ele", sort_array("tri_ele_origin", False)).drop("tri_ele_origin") # sort_array("tri", False), False means descending order

df_tri_order.printSchema()
df_tri_order.rdd.getNumPartitions()
#df_tri_order.show()

root
 |-- tri_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- tri: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- tri_ele: array (nullable = false)
 |    |-- element: float (containsNull = true)



200

##### globally get VT relation

In [17]:
# union and group vertices and get the preliminary VT relation
def grp_union(df_tri_order):
    '''
    df_tri_order: a DataFrame storing sorted extreme vertices of each triangle
    '''
    # groupby and collect_set
    df_tri_group1 = df_tri_order.groupBy('r1','r1_ele').agg(collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
    df_tri_group2 = df_tri_order.groupBy('r2','r2_ele').agg(collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
    df_tri_group3 = df_tri_order.groupBy('r3', 'r3_ele').agg(collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
    
    union12 = df_tri_group1.union(df_tri_group2) # the title of union12 will be the same as df_sj_group1
    union123 = union12.union(df_tri_group3) # the title of union123 will be the same as union12, so as df_sj_group1

    union123_group = union123.groupBy('r1','r1_ele').agg(collect_list('multi_tri').alias('multi_tri_list'), collect_list('multi_tri_ele').alias('multi_tri_ele_list'))
    
    # print("the schema of union123_group:", union123_group.printSchema())
    return union123_group

In [18]:
union123_group = grp_union(df_tri_order)
union123_group.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- multi_tri_list: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |-- multi_tri_ele_list: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: float (containsNull = true)



In [19]:
# sort the triangles in the preliminary VT relation
def get_multi_tri_order(tri_list, tri_ele_list):
    '''
    get_multi_tri_order is used to obtain the adjacent triangles, the results are in ascending order, but the triangle itself is in descending
    e.g., [[3, 2, 1], [5, 3, 2], [5, 3, 1]]
    tri_list: an array of array of array of integer
    tri_ele_list: an array of array of array of float
    '''

    tri = []
    tri_ele = []
    # pt_list.append(pt_self) # if we don't calculate roughness, we don't need self vertex
    for i in range(len(tri_list)):
        for j in range(len(tri_list[i])):
            # if tri_list[i][j] not in tri: # we do not need to check if it exists in tri_list, because we can prove that each one is unique
            tri.append(tri_list[i][j]) # update() will add multiple elements to a set, while add() only add one element to a set   
            tri_ele.append(tri_ele_list[i][j])
    
    tri_copy = copy.deepcopy(tri) # deep copy tri
    tri.sort() # sort the list of array, e.g., tri=[[3, 2, 1], [5, 2, 1], [5, 3, 1]], after sorting, [[3, 2, 1], [5, 2, 1], [5, 3, 1]]
    
    tri_ele_sort = []
    # sort tri_ele according to the same rule as tri
    for i in range(len(tri)):
        index_in_tri_origin = tri_copy.index(tri[i])
        tri_ele_sort.append(tri_ele[index_in_tri_origin])
    
    return tri, tri_ele_sort

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html

# StructType for get_multi_tri_order        
get_multi_tri_order_schema = StructType([
    StructField("multi_tri_sort", ArrayType(ArrayType(IntegerType())),True), 
    StructField('multi_tri_ele_sort',ArrayType(ArrayType(FloatType())),True)
])

get_multi_tri_order_udf = udf(get_multi_tri_order, get_multi_tri_order_schema)

df_VT = union123_group.withColumn("multi_tri_order", get_multi_tri_order_udf(union123_group.multi_tri_list, union123_group.multi_tri_ele_list))
df_VT = df_VT.select(col("r1").alias("Ver"), col("multi_tri_order.multi_tri_sort").alias("VT_filtra"), col("multi_tri_order.multi_tri_ele_sort").alias("VT_ele"))
df_VT.printSchema()
print("number of partitions for df_VT:", df_VT.rdd.getNumPartitions())

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VT_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)

number of partitions for df_VT: 200


##### globally get VV relation

In [20]:
# function to get VV using groupby
def get_VV(df_tri_order):
    df_VV_init_1 = df_tri_order.select("r1", "r2", "r3")
    df_VV_init_2 = df_tri_order.select("r2", "r1", "r3")
    df_VV_init_3 = df_tri_order.select("r3", "r1", "r2")
    
    df_VV_union12 = df_VV_init_1.union(df_VV_init_2)
    df_VV_union123 = df_VV_union12.union(df_VV_init_3)
    
    # df_VV_union123.cache()
    
    # df_VV_union123.persist(StorageLevel.MEMORY_ONLY)
    # df_VV_union123.printSchema()
    
    df_VV_gp = df_VV_union123.groupBy('r1').agg(collect_set('r2').alias('multi_r2_set'),collect_set('r3').alias('multi_r3_set'))
    
    return df_VV_gp

In [21]:
df_VV_gp = get_VV(df_tri_order)

# df_VV_gp.cache()

df_VV_gp.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- multi_r2_set: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_r3_set: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [22]:
# obtain VV relation
def get_multi_pt_order(pt_list1, pt_list2):
# get_multi_pt_index is used to obtain the adjacent vertexes index, including the vertex itself
# pt_list1: partial adjacent vertex indexes of join result 1
# pt_list2: partial adjacent vertex indexes of join result 2
    pt_init = pt_list1 + pt_list2
    pt_set = set(pt_init)
    
    pt_list = sorted(pt_set) # sorted(pt_set, reverse=False), False in ascending order while True in descending order
    
    return pt_list # directly return a list, e.g., [0, 1, 5, 6], the length of returned column list will be 12, incluing "[" and empty char

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_multi_pt_order_udf = udf(get_multi_pt_order, ArrayType(IntegerType()))

In [23]:
df_VV = df_VV_gp.withColumn("VV", get_multi_pt_order_udf(df_VV_gp.multi_r2_set, df_VV_gp.multi_r3_set)).drop('multi_r2_set', 'multi_r3_set')

df_VV = df_VV.withColumnRenamed('r1', 'Ver')

df_VV.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)



##### globally get VE relation

In [24]:
# function to get VE from DF_T
def get_VE_init(df_tri_order):
    df_VE_1 = df_tri_order.withColumn("e1", sort_array(F.array("r1", "r2"), False)).withColumn("e2", sort_array(F.array("r1", "r3"), False)).drop('r2', 'r3')
    df_VE_2 = df_tri_order.withColumn("e1", sort_array(F.array("r1", "r2"), False)).withColumn("e2", sort_array(F.array("r2", "r3"), False)).drop('r1', 'r3')
    df_VE_3 = df_tri_order.withColumn("e1", sort_array(F.array("r2", "r3"), False)).withColumn("e2", sort_array(F.array("r1", "r3"), False)).drop('r1', 'r2')
    
    df_VE_union12 = df_VE_1.union(df_VE_2)
    df_VE_union123 = df_VE_union12.union(df_VE_3)
    
    df_VE_init = df_VE_union123.groupBy('r1').agg(collect_set('e1').alias('multi_e1'), collect_set('e2').alias('multi_e2'))
    return df_VE_init

df_VE_init = get_VE_init(df_tri_order)

# df_VE_init.cache()

df_VE_init.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- multi_e1: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_e2: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [25]:
# obtain VV relation
def get_VE(multi_e1, multi_e2):
# get_VE is used to obtain the partial VE relation
# multi_e1: partial VE relation
# multi_e2: partial VE relation
    edges = set()
    for e in multi_e1:
        edges.add(tuple(e))
        
    for e in multi_e2:
        edges.add(tuple(e))
    
    edges_list = sorted(edges) # sorted(pt_set, reverse=False), False in ascending order while True in descending order
    
    return edges_list

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_VE_udf = udf(get_VE, ArrayType(ArrayType(IntegerType())))

df_VE = df_VE_init.withColumn("VE", get_VE_udf(df_VE_init.multi_e1, df_VE_init.multi_e2)).drop('multi_e1', 'multi_e2')
df_VE = df_VE.withColumnRenamed('r1', 'Ver')

df_VE.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VE: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [26]:
df_VT_VV = df_VT.join(df_VV, df_VT.Ver == df_VV.Ver)
df_VT_VV = df_VT_VV.select(df_VT.Ver, "VT_filtra", "VT_ele", "VV")

df_VT_VV.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VT_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [27]:
df_VT_VV_VE = df_VT_VV.join(df_VE, df_VT_VV.Ver == df_VE.Ver)
df_VT_VV_VE = df_VT_VV_VE.select(df_VT_VV.Ver, "VT_filtra", "VT_ele", "VV", "VE")

df_VT_VV_VE.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VT_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- VE: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



##### compute the lower star

In [28]:
# obtain Lower Star
def get_LS(Ver, VT_filtra, VT_ele, VV):
    # get_LS is used to obtain the VV and VE relation and compute the lower star
    
    # VE relation
    LS_edge = []
    for i in range(len(VV)):
        if VV[i] < int(Ver):
            LS_edge.append([int(Ver),VV[i]])
        
    LS_tri = []
    LS_tri_ele = []
    
    for i in range(len(VT_filtra)):
        if Ver >= max(VT_filtra[i]): # Ver_ele is not the maximum vertex
            LS_tri.append(VT_filtra[i])
            LS_tri_ele.append(VT_ele[i])
                
    return LS_edge, LS_tri, LS_tri_ele

# StructType for get_multi_tri_order        
get_LS_schema = StructType([
    StructField("LS_edge", ArrayType(ArrayType(IntegerType())),True), 
    StructField("LS_tri_filtra", ArrayType(ArrayType(IntegerType())),True),
    StructField('LS_tri_ele',ArrayType(ArrayType(FloatType())),True)
])

get_LS_udf = udf(get_LS, get_LS_schema)

df_LS = df_VT_VV_VE.withColumn("LS", get_LS_udf(df_VT_VV_VE.Ver, df_VT_VV_VE.VT_filtra, df_VT_VV_VE.VT_ele, df_VT_VV_VE.VV))
df_LS = df_LS.select("Ver", "VT_filtra", col("LS.LS_edge").alias("LS_edge"), col("LS.LS_tri_filtra").alias("LS_tri_filtra"), col("LS.LS_tri_ele").alias("LS_tri_ele"))
df_LS.printSchema()
# df_LS_edge.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_edge: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_tri_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_tri_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)



### compute Forman Gradient

In [29]:
from collections import deque

In [30]:
# get the Forman gradient from lower stars
def get_Forman(Ver, LS_edge, LS_filtra, LS_ele):
    '''
    get_Forman is used to compute Forman gradient
    LS_edge is triangles of lower star in ascending order
    LS_tri is triangles of lower star in ascending order
    '''
    # inner udf to get the number of unpaired edges for a triangle
    def num_unpaired_edges(tri_inner, crit_cell_inner, Forman_vec_pair_VE_inner, Forman_vec_pair_ET_inner):
        # compute the number of unpaired edges
        edge0 = [tri_inner[0], tri_inner[1]]
        edge1 = [tri_inner[0], tri_inner[2]]
        
        num_unpaired_edge = 2
        paired_edge = []
        for crit_inner_temp in crit_cell_inner:
            if len(crit_inner_temp) == 2: # crit_inner_temp is a critical edge
                paired_edge.append(crit_inner_temp)
                
        for pair_vec in Forman_vec_pair_VE_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, tri)
            paired_edge.append(pair_vec[1])
            # if type(pair_vec[0]) != int and len(pair_vec[0]) == 2: # pair_vec[0] is a paired edge, like in (edge, tri)
                # paired_edge.append(pair_vec[0])
            # if len(pair_vec[1]) == 2: # pair_vec[1] is a paired edge, like in (ver, edge)
                # paired_edge.append(pair_vec[1])
        for pair_vec in Forman_vec_pair_ET_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, tri)
            paired_edge.append(pair_vec[0])
                
        if edge0 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
        if edge1 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
            
        return num_unpaired_edge
    
    # inner udf to get the edge which is paired with a triangles
    def pair_an_edge_from_PQzero(alpha_inner, PQzero_inner):
    # alpha_inner is a triangle in [r0, r1, r2] format, and the vertext's elevation is in descending order
    # PQzero_inner is a queue storing all other edges
        alpha_e0 = [alpha_inner[0], alpha_inner[1]]
        alpha_e1 = [alpha_inner[0], alpha_inner[2]]
        for edge_inner in PQzero_inner:
            if edge_inner == alpha_e0:
                return edge_inner
            if edge_inner == alpha_e1:
                return edge_inner
            
    crit_cell = [] # crit_cell will store critical simplices
    # Forman_vec_pair = [] # Forman_vec_pair will store all Forman gradient pairs as a vector
    Forman_vec_pair_VE = []
    Forman_vec_pair_ET = []
    if (len(LS_edge)+len(LS_filtra)) == 0: # ver is a local minimum
        crit_cell.append([Ver]) # store minimum as an array [Ver] though it has only one element
    elif len(LS_edge) > 0:
        Forman_vec_pair_VE.append([Ver, LS_edge[0]]) # LS_edge[0] is the minimum edge since it is in ascending order
        
        # define two queues, PQzero storing all other edges, PQone storing all triangles which have only one unpaired edge
        PQzero = deque()
        PQone = deque()
        PQone_ele = deque()
        for i in range(1, len(LS_edge)): # add the other edges to PQzero
            PQzero.append(LS_edge[i])
            
        cell_temp_index_of_remove = []
        index_temp = 0
        for cell_temp in LS_filtra:
            if num_unpaired_edges(cell_temp, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 1:
                PQone.append(cell_temp) # storing cells with one unpaired edges, which are triangles in TIN
                PQone_ele.append(LS_ele[index_temp])
                # remove cell_temp from LS_filtra
                # LS_filtra.remove(cell_temp)
                cell_temp_index_of_remove.append(index_temp)
            index_temp += 1
            
        # remove cell_temp from LS_filtra and LS_ele
        for i in reversed(cell_temp_index_of_remove):
            del LS_filtra[i]
            del LS_ele[i]
                
        # sort PQone in ascending order, the initial queue is already sorted when creating
        # PQone = sorted(PQone) # sort() is not supported but we can use sorted()
        # PQone = deque(PQone) # after sorting, PQone one will be a list instead of a queue, we need to reconstruct this queue
        while len(PQone) > 0 or len(PQzero) > 0:
            while len(PQone) > 0:
                # alpha = PQone.popleft() # alpha is a triangle in the PQone
                # alpha should be a triangle with lower elevation when PQone have more than one triangle
                if len(PQone) > 1 and abs(PQone_ele[0][1]-PQone_ele[1][1]) < 0.000001 and PQone_ele[0][2] > PQone_ele[1][2]:
                    alpha = PQone[1] # alpha is the second triangle
                    del PQone[1]
                    del PQone_ele[1]
                else:
                    alpha = PQone.popleft() # alpha is the first triangle, which is the same as alpha=PQone[0], then del PQone[0]
                    del PQone_ele[0]
                
                if num_unpaired_edges(alpha, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 0:
                    PQzero.append(alpha)
                else:
                    pair_alpha = pair_an_edge_from_PQzero(alpha, PQzero) # pair_alpha is an edge, [pair_alpha, alpha] will be a new paired vector
                    # if pair_alpha in PQzero:
                    PQzero.remove(pair_alpha) # remove pair_alpha from PQzero
                    # Forman_vec_pair.append([pair_alpha, alpha]) # add the [pair_alpha, alpha], which is [edge, tri]
                    Forman_vec_pair_ET.append([pair_alpha, alpha]) # add the [pair_alpha, alpha], which is [edge, tri]
                    
                    beta_cell_temp_index_of_remove = []
                    beta_index_temp = 0
                    for cell_beta in LS_filtra: # this can be optimized
                        if cell_beta not in PQone and num_unpaired_edges(cell_beta, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 1:
                            PQone.append(cell_beta)
                            PQone_ele.append(LS_ele[beta_index_temp])
                            # remove cell_beta from LS_tri
                            # LS_tri.remove(cell_beta)
                            beta_cell_temp_index_of_remove.append(beta_index_temp)
                        beta_index_temp += 1
                    
                    # remove cell_beta from LS_filtra and LS_ele
                    for i in reversed(beta_cell_temp_index_of_remove):
                        del LS_filtra[i]
                        del LS_ele[i]
                    
                    PQone = sorted(PQone)
                    PQone = deque(PQone) # after sorting, PQone one will be a list instead of a queue, we need to reconstruct this queue
            if len(PQzero) > 0:
                gama = PQzero.popleft() # gama is a critical simplex
                crit_cell.append(gama)
                
                gama_cell_temp_index_of_remove = []
                gama_index_temp = 0
                for cell_gama in LS_filtra: # this can be optimized
                    if cell_gama not in PQone and num_unpaired_edges(cell_gama, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 1:
                        PQone.append(cell_gama)
                        PQone_ele.append(LS_ele[gama_index_temp])
                        # remove cell_gama from LS_tri
                        # LS_tri.remove(cell_gama)
                        gama_cell_temp_index_of_remove.append(gama_index_temp)
                    gama_index_temp += 1
                    
                # remove cell_beta from LS_filtra and LS_ele
                for i in reversed(gama_cell_temp_index_of_remove):
                    del LS_filtra[i]
                    del LS_ele[i]
                        
                PQone = sorted(PQone)
                PQone = deque(PQone) # after sorting, PQone one will be a list instead of a queue, we need to reconstruct this queue
                
    return crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET

# StructType for Crit_cell        
add_Crit_cell_schema = StructType([
    StructField('Crit_cell',ArrayType(IntegerType()),True)
])

# StructType for VE pairs        
add_VE_schema = StructType([
    StructField("VE_pair_V", IntegerType(),True), 
    StructField('VE_pair_E',ArrayType(IntegerType()),True)
])

# StructType for ET pairs        
add_multi_ET_schema = StructType([
    StructField("ET_pair_E", ArrayType(IntegerType()),True), 
    StructField('ET_pair_T',ArrayType(IntegerType()),True)
])
 
# the whole StructType
get_Forman_schema = StructType([
    StructField("Crit_cell", ArrayType(ArrayType(IntegerType())), True),
    StructField("VE_pair", ArrayType(add_VE_schema), True),
    StructField("ET_pair", ArrayType(add_multi_ET_schema), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_Forman_udf = udf(get_Forman, get_Forman_schema)

### OR get Forman gradient from previously saved results

In [28]:
file_df_Forman = directory + '/' + tin_filename + '_sortForman' + '.parquet'
df_Forman = spark.read.parquet(file_df_Forman)
df_Forman.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Forman: struct (nullable = true)
 |    |-- Crit_cell: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- VE_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- VE_pair_V: integer (nullable = true)
 |    |    |    |-- VE_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- ET_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- ET_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |    |    |-- ET_pair_T: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)



# Extract critical vertices, edeges, and triangles

In [29]:
df_crit = df_Forman.select("Ver", col("Forman.Crit_cell").alias("Critical"))
print("number of partitions for df_crit:", df_crit.rdd.getNumPartitions())
df_crit.printSchema()
# df_crit.show()

number of partitions for df_crit: 200
root
 |-- Ver: integer (nullable = true)
 |-- Critical: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [30]:
# split the Critical column to obtain the critical vertices, edges, and triangles
def get_crit_VET(crit_cell):
    if len(crit_cell) < 1: # crit_cell is empty, there are no critical simplices within this lower star
        return
    else:
        # initialize the critical vertices, critical edges, and critical triangles
        crit_ver = None
        crit_edge = []
        crit_tri = []
        for icrit_cell_temp in crit_cell:
            if len(icrit_cell_temp) == 1: # icrit_cell_temp stores a critical vertex
                crit_ver = icrit_cell_temp[0]
            if len(icrit_cell_temp) == 2: # icrit_cell_temp stores critical edges
                crit_edge.append(icrit_cell_temp)
            if len(icrit_cell_temp) == 3: # icrit_cell_temp stores critical triangles
                crit_tri.append(icrit_cell_temp)
        if len(crit_edge) == 0:
            crit_edge = None
        if len(crit_tri) == 0:
            crit_tri = None
        return crit_ver, crit_edge, crit_tri
        
# the whole StructType
get_crit_VET_schema = StructType([
    StructField("crit_ver", IntegerType(), True),
    StructField("crit_edge", ArrayType(ArrayType(IntegerType())), True),
    StructField("crit_tri", ArrayType(ArrayType(IntegerType())), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_crit_VET_udf = udf(get_crit_VET, get_crit_VET_schema)

df_crit_VET = df_crit.withColumn("crit_cell", get_crit_VET_udf(df_crit.Critical))
print("number of partitions for df_crit_VET:", df_crit_VET.rdd.getNumPartitions())
df_crit_VET.printSchema()
# df_crit_VET.show()

number of partitions for df_crit_VET: 200
root
 |-- Ver: integer (nullable = true)
 |-- Critical: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- crit_cell: struct (nullable = true)
 |    |-- crit_ver: integer (nullable = true)
 |    |-- crit_edge: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- crit_tri: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)



In [31]:
# to get critical vertices
df_crit_V = df_crit_VET.select(col("Ver"), col("crit_cell.crit_ver").alias("Min_ver")).na.drop()
df_crit_V.printSchema()
print("number of partitions for df_crit_V:", df_crit_V.rdd.getNumPartitions())
# df_crit_V.show()

root
 |-- Ver: integer (nullable = true)
 |-- Min_ver: integer (nullable = true)

number of partitions for df_crit_V: 200


In [32]:
# num_crit_V = df_crit_V.count()
# print("number of critical vertices:", num_crit_V)

In [33]:
# to get critical edges
df_crit_E_init = df_crit_VET.select(col("Ver"), col("crit_cell.crit_edge").alias("Saddle_edge_init")).na.drop()

# there may be multiple saddles inside one lower star
df_crit_E = df_crit_E_init.select(col("Ver"), explode("Saddle_edge_init").alias("Saddle_edge")).na.drop()
df_crit_E.printSchema()
print("number of partitions for df_crit_E:", df_crit_E.rdd.getNumPartitions())
# df_crit_E.show()

root
 |-- Ver: integer (nullable = true)
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)

number of partitions for df_crit_E: 200


In [34]:
# num_crit_E = df_crit_E.count()
# print("number of critical edges:", num_crit_E)

In [35]:
# to get critical triangles
df_crit_T_init = df_crit_VET.select(col("Ver"), col("crit_cell.crit_tri").alias("Max_tri_init")).na.drop()

# there could be multiple critical triangles within one lower star
df_crit_T = df_crit_T_init.select(col("Ver"), explode("Max_tri_init").alias("Max_tri")).na.drop()
df_crit_T.printSchema()
print("number of partitions for df_crit_T:", df_crit_T.rdd.getNumPartitions())
# df_crit_T.show()

root
 |-- Ver: integer (nullable = true)
 |-- Max_tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)

number of partitions for df_crit_T: 200


In [36]:
# num_crit_T = df_crit_T.count()
# print("number of critical triangles:", num_crit_T)

# Methods to build Graph VE

In [40]:
# to obtain critical simplices and VE_paris from the Forman gradient
df_Forman_VE_pairs = df_Forman.select(col("Ver"), col("Forman.Crit_cell").alias("Crit_cell"), col("Forman.VE_pair").alias("VE_pair"))
df_Forman_VE_pairs.printSchema()
# df_Forman_VE_pairs.show()

root
 |-- Ver: integer (nullable = true)
 |-- Crit_cell: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VE_pair: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- VE_pair_V: integer (nullable = true)
 |    |    |-- VE_pair_E: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [41]:
'''
# construct a graph from VE_pairs and critical simplices
def Build_V1(VE_pair):
    if len(VE_pair) > 0:
        return VE_pair[0][1] # VE_pair[0] is a VE pair, VE_pair[0][1] is an edge
    
# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
Build_V1_udf = udf(Build_V1, ArrayType(IntegerType()))
df_graph_V1 = df_Forman_VE_pairs.withColumn("V1_edge", Build_V1_udf(df_Forman_VE_pairs.VE_pair)).select(col("Ver").alias("V1_node"), "V1_edge")
df_graph_V1.printSchema()
# df_graph_V1.show()
'''

'\n# construct a graph from VE_pairs and critical simplices\ndef Build_V1(VE_pair):\n    if len(VE_pair) > 0:\n        return VE_pair[0][1] # VE_pair[0] is a VE pair, VE_pair[0][1] is an edge\n    \n# convert a function to an udf and determine the return type\n# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html\nBuild_V1_udf = udf(Build_V1, ArrayType(IntegerType()))\ndf_graph_V1 = df_Forman_VE_pairs.withColumn("V1_edge", Build_V1_udf(df_Forman_VE_pairs.VE_pair)).select(col("Ver").alias("V1_node"), "V1_edge")\ndf_graph_V1.printSchema()\n# df_graph_V1.show()\n'

In [42]:
# to get the DataFrame containg all vertices and VE pairs 
df_graph_V1 = df_Forman_VE_pairs.select(col("Ver").alias("V1_node"), df_Forman_VE_pairs.VE_pair.VE_pair_E.alias("V1_edge"))
df_graph_V1.printSchema()

root
 |-- V1_node: integer (nullable = true)
 |-- V1_edge: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



#### prepare nodes of the Graph VE

In [43]:
df_graph_V1_node = df_graph_V1.select(col("V1_node").alias("id"))
df_graph_V1_node.printSchema()
# df_graph_V1_node.show()

root
 |-- id: integer (nullable = true)



##### repartition and save df_graph_V1_node

In [44]:
# df_graph_V1_node = df_graph_V1_node.repartition(20) date_name
t_save_node_0 = time.time()

file_df_graph_V1_node = 'SigSpatial_data' + '/' + tin_filename + '_VE_node' + '_' + date_name + '.parquet'
# df_graph_V1_node.write.parquet(file_df_graph_V1_node) 

t_save_node_1 = time.time()
print("Time cost to save Graph_VE nodes in Parquet:", t_save_node_1 - t_save_node_0)

Time cost to save Graph_VE nodes in Parquet: 0.00010037422180175781


#### prepare arcs of the Graph VE

In [45]:
df_graph_V1_arc = df_graph_V1.select(df_graph_V1.V1_edge[0][0].alias('src'), df_graph_V1.V1_edge[0][1].alias('dst')).na.drop()
df_graph_V1_arc.printSchema()
print("number of partitions for df_graph_V1_arc:", df_graph_V1_arc.rdd.getNumPartitions())
# df_graph_V1_arc.show(30)

root
 |-- src: integer (nullable = true)
 |-- dst: integer (nullable = true)

number of partitions for df_graph_V1_arc: 200


##### repartition and save the dataframe df_graph_V1_arc

In [46]:
# df_graph_V1_arc = df_graph_V1_arc.repartition(20)
# print("number of partitions for df_graph_V1_arc:", df_graph_V1_arc.rdd.getNumPartitions())

# save Graph_VE arcs to Parquet
t_save_arc_0 = time.time()
file_df_graph_V1_arc = 'SigSpatial_data' + '/' + tin_filename + '_VE_arc' + '_' + date_name + '.parquet'
# df_graph_V1_arc.write.parquet(file_df_graph_V1_arc) 

t_save_arc_1 = time.time()
print("Time cost to save Graph_VE arcs in Parquet:", t_save_arc_1 - t_save_arc_0)

Time cost to save Graph_VE arcs in Parquet: 9.1552734375e-05


##### OR get nodes and arcs from previously saved results

In [47]:
# read nodes of V1
schema_df_graph_V1_node_final = StructType([ \
    StructField("id",IntegerType(),True)
  ])
file_df_graph_V1_node_final_hdfs = 'SigSpatial_data' + '/' + tin_filename + '_graph_V1_node.parquet'
df_graph_V1_node = spark.read.format("parquet") \
      .option("header", False) \
      .schema(schema_df_graph_V1_node_final)\
      .load(file_df_graph_V1_node_final_hdfs)
print("Number of partitions for df_graph_V1_node before repartition:", df_graph_V1_node.rdd.getNumPartitions())
df_graph_V1_node.printSchema()

# read arcs of V1
schema_df_graph_V1_edge_final = StructType([ \
    StructField("src",IntegerType(),True), \
    StructField("dst",IntegerType(),True)
  ])
file_df_graph_V1_edge_final_hdfs = 'SigSpatial_data' + '/' + tin_filename + '_graph_V1_arc.parquet'
df_graph_V1_arc = spark.read.format("parquet") \
      .option("header", False) \
      .schema(schema_df_graph_V1_edge_final)\
      .load(file_df_graph_V1_edge_final_hdfs)
print("Number of partitions for df_graph_V1_arc before repartition:", df_graph_V1_arc.rdd.getNumPartitions())
df_graph_V1_arc.printSchema()

Number of partitions for df_graph_V1_node before repartition: 80
root
 |-- id: integer (nullable = true)

Number of partitions for df_graph_V1_arc before repartition: 120
root
 |-- src: integer (nullable = true)
 |-- dst: integer (nullable = true)



### construct Graph_VE

In [48]:
graph_VE = GraphFrame(df_graph_V1_node, df_graph_V1_arc)

In [49]:
# set a checkpoint directory to improve performance
# Checkpointing regularly helps recover from failures, clean shuffle files, shorten the
# lineage of the computation graph, and reduce the complexity of plan optimization.

spark.sparkContext.setCheckpointDir('checkpoints')

### Compute influence regions of minima, which are the connected components of Graph_VE

In [50]:
t_con_0 = time.time()
# result_con consists of two columns, vertex id, component
result_con = graph_VE.connectedComponents()
result_con.printSchema()

t_con_1 = time.time()
t_con = t_con_1 - t_con_0
print("time cost of connected components:",t_con)

root
 |-- id: integer (nullable = true)
 |-- component: long (nullable = true)

time cost of connected components: 149.9858365058899


##### 1) left join result_con with saddles to identify the boundary vertices of saddles that belong to the same component

In [51]:
# obtain the saddle pts
df_crit_E2pts = df_crit_E.select(explode("Saddle_edge").alias("Saddle_pts"))
# some nodes may belong to multiple saddles, we need to remove them
df_crit_E2pts = df_crit_E2pts.distinct()
df_crit_E2pts.printSchema()

root
 |-- Saddle_pts: integer (nullable = true)



In [52]:
# OR get df_crit_E2pts from previously saved results

schema_df_crit_E2pts = StructType([ \
    StructField("Saddle_pts",IntegerType(),True)
  ])
file_df_crit_E2pts_hdfs = 'hdfs_data' + '/' + tin_filename + '_crit_E2pts.parquet'
df_crit_E2pts_file = spark.read.format("parquet") \
      .option("header", False) \
      .schema(schema_df_crit_E2pts)\
      .load(file_df_crit_E2pts_hdfs)

df_crit_E2pts_file.printSchema()

root
 |-- Saddle_pts: integer (nullable = true)



In [53]:
# broadcast left join connected components with saddles
# result_con contains the df_graph_V1_node
result_con_saddle = result_con.join(df_crit_E2pts,result_con.id==df_crit_E2pts.Saddle_pts, "inner")
result_con_saddle = result_con_saddle.withColumn("att", when(result_con_saddle.Saddle_pts.isNull() == False, result_con_saddle.Saddle_pts).otherwise(-1)).drop('Saddle_pts')

unique_SdlPts_per_con = result_con_saddle.groupBy('component').agg(collect_list('id').alias('multi_SdlPts'))

result_con_saddle.printSchema()

root
 |-- id: integer (nullable = true)
 |-- component: long (nullable = true)
 |-- att: integer (nullable = true)



##### 2) left join df_graph_V1_arc with result_con to identify the arcs that belong to the connected component

In [54]:
# left join df_graph_V1_arc with result_con to identify which component that each arc belongs to
df_G_V1_arc_component = df_graph_V1_arc.join(result_con,df_graph_V1_arc.src==result_con.id, "left").drop('id')
df_G_V1_arc_component.printSchema()

root
 |-- src: integer (nullable = true)
 |-- dst: integer (nullable = true)
 |-- component: long (nullable = true)



In [55]:
Num_subgraphs = 20
t_find_minmax_0 = time.time()

# Find the minimum and maximum values of "component" in result_con
sub_graph_tmp_min = result_con.agg({"component": "min"}).collect()[0][0]
sub_graph_tmp_max = result_con.agg({"component": "max"}).collect()[0][0]

sub_graph_list = []
for i in range(Num_subgraphs+1):
    sub_graph_tmp = int(i * (sub_graph_tmp_max - sub_graph_tmp_min)/(Num_subgraphs) + sub_graph_tmp_min)
    sub_graph_list.append(sub_graph_tmp)
    
'''

# sort and collect result_con_unique
result_con_unique = result_con.groupBy('component').count()
# result_con_unique.printSchema()
# result_con_unique.show()

result_con_unique = result_con_unique.sort(col('component'))
result_con_unique_col = result_con_unique.collect()

sub_graph_list = []

for i in range(Num_subgraphs):
    tmp = int(i / (Num_subgraphs-1) * (len(result_con_unique_col) -1))
    sub_graph_tmp = result_con_unique_col[tmp]['component']
    sub_graph_list.append(sub_graph_tmp)
'''    

t_find_minmax_1 = time.time()
t_find_minmax = t_find_minmax_1 - t_find_minmax_0
print("time cost to find sub_graph_list:", t_find_minmax)

time cost to find sub_graph_list: 6.743970632553101


In [56]:
# Create empty DataFrame with a schema
schema_special = StructType([
    StructField('id', IntegerType(),True),
    StructField("component", IntegerType(),True),
    StructField("att", IntegerType(),True),   
    StructField("label", IntegerType(),True)   
])

df_empty = spark.createDataFrame([], schema_special)

In [57]:
# construct subgraphs and union the LPA results
t_LPA_total_0 = time.time()

sub_time = [] # to store time cost for each subgraph
for i in range(len(sub_graph_list) - 1):
    t_LPA_sub_0 = time.time()
    sub_graph_tmp_0 = sub_graph_list[i]
    sub_graph_tmp_1 = sub_graph_list[i+1]
    
    if i == len(sub_graph_list) - 1: # the last subgraphs should contain sub_graph_tmp_max
        sub_graph_node = result_con_saddle.filter((result_con_saddle.component >= sub_graph_tmp_0) & (result_con_saddle.component <= sub_graph_tmp_1))
        sub_graph_arc = df_G_V1_arc_component.filter((df_G_V1_arc_component.component >= sub_graph_tmp_0) & (df_G_V1_arc_component.component <= sub_graph_tmp_1)).drop('id')
    else:
        sub_graph_node = result_con_saddle.filter((result_con_saddle.component >= sub_graph_tmp_0) & (result_con_saddle.component < sub_graph_tmp_1))
        # sub_graph_node.printSchema() # 3 columns: id, component, att
        sub_graph_arc = df_G_V1_arc_component.filter((df_G_V1_arc_component.component >= sub_graph_tmp_0) & (df_G_V1_arc_component.component < sub_graph_tmp_1)).drop('id')
        # sub_graph_arc.printSchema() # 3 columns: src, dst, component
    
    sub_graph = GraphFrame(sub_graph_node, sub_graph_arc)
    
    sub_graph_LPA = sub_graph.pregel \
         .setMaxIter(10) \
         .withVertexColumn("label", col("att"), \
             Pregel.msg()) \
         .sendMsgToDst(when(Pregel.dst("label")>0, Pregel.dst("label")).otherwise(Pregel.src("label"))) \
         .aggMsgs(F.max(Pregel.msg())) \
         .run()
    t_LPA_sub_1 = time.time()
    df_empty = df_empty.union(sub_graph_LPA)
    print("Time cost for LPA to subgraph %ld: %f" % (sub_graph_tmp_0, t_LPA_sub_1 - t_LPA_sub_0))
    sub_time.append(t_LPA_sub_1 - t_LPA_sub_0)
    
t_LPA_total_1 = time.time()
print("*******************************************************")
print("Total time cost for LPA:", t_LPA_total_1 - t_LPA_total_0)

Time cost for LPA to subgraph 0: 162.915565
Time cost for LPA to subgraph 2449372: 155.676687
Time cost for LPA to subgraph 4898744: 154.333997
Time cost for LPA to subgraph 7348116: 153.238565
Time cost for LPA to subgraph 9797489: 143.981099
Time cost for LPA to subgraph 12246861: 150.578045
Time cost for LPA to subgraph 14696233: 147.890140
Time cost for LPA to subgraph 17145605: 153.646286
Time cost for LPA to subgraph 19594978: 146.937313
Time cost for LPA to subgraph 22044350: 141.634086
Time cost for LPA to subgraph 24493722: 145.904270
Time cost for LPA to subgraph 26943094: 148.900878
Time cost for LPA to subgraph 29392467: 150.281316
Time cost for LPA to subgraph 31841839: 151.051762
Time cost for LPA to subgraph 34291211: 148.991852
Time cost for LPA to subgraph 36740583: 148.250391
Time cost for LPA to subgraph 39189956: 144.659430
Time cost for LPA to subgraph 41639328: 144.304403
Time cost for LPA to subgraph 44088700: 144.043732
Time cost for LPA to subgraph 46538072: 15

In [58]:
print("time cost of connected components:",t_con)

print("time cost to find sub_graph_list:", t_find_minmax)

print("Total time cost for LPA:", t_LPA_total_1 - t_LPA_total_0)

time cost of connected components: 149.9858365058899
time cost to find sub_graph_list: 6.743970632553101
Total time cost for LPA: 2990.0800545215607


## Compute ascending 2-manifolds (influence regions of minima), which are the connected components of Graph_VE

In [59]:
t_con_0 = time.time()
# result_con consists of two columns, vertex id, component
result_con = graph_VE.connectedComponents()
result_con.printSchema()

t_con_1 = time.time()
t_con = t_con_1 - t_con_0
print("time cost of connected components:",t_con)

root
 |-- id: integer (nullable = true)
 |-- component: long (nullable = true)

time cost of connected components: 88.56818056106567


# Methods to build Graph ET

#### prepare nodes of Graph ET

In [36]:
# all triangles are nodes of graph V2
df_graph_V2_node = df_tri_order.select(col("tri_order"), col("tri"))
df_graph_V2_node.printSchema()
#df_graph_V2_node.show()

root
 |-- tri_order: integer (nullable = true)
 |-- tri: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [37]:
# add a special triangle [-1, -1, -1] as node -1; this is useful to deal with boundary elements
# e.g., for a boundary edge, it has only one coboundary triangle; to guarantee a consistent encoding, each edge has two coboundary triangels, the other coboundary triangle for a boundary edge is [-1,-1,-1]
# columns_special = ['tri', 'V2_node']
value_special = [(-1, [-1, -1, -1])]
schema_special = StructType([
    StructField('tri_order', IntegerType(),True),
    StructField("tri", ArrayType(IntegerType()),True)    
])
df_special_V2_node = spark.createDataFrame(value_special, schema_special)
df_special_V2_node.printSchema()
#df_special_V2_node.show()

root
 |-- tri_order: integer (nullable = true)
 |-- tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)



##### nodes of Graph_ET

In [38]:
df_graph_V2_node = df_graph_V2_node.union(df_special_V2_node)
df_graph_V2_node_final = df_graph_V2_node.select(col("tri_order").alias("id"))
df_graph_V2_node_final.printSchema()
#df_graph_V2_node_final.show()

root
 |-- id: integer (nullable = true)



In [39]:
# df_graph_V2_node = df_graph_V2_node.repartition(20) date_name
t_save_node_0 = time.time()

file_df_graph_V2_node = 'SigSpatial_data' + '/' + tin_filename + '_ET_node' + '_' + '03132024' + '.parquet'
# df_graph_V2_node_final.write.parquet(file_df_graph_V2_node) 

t_save_node_1 = time.time()
print("Time cost to save Graph_ET nodes in Parquet:", t_save_node_1 - t_save_node_0)

Time cost to save Graph_ET nodes in Parquet: 6.628036499023438e-05


#### prepare arcs of Graph_ET

In [40]:
# Method: firstly extract df_Forman_ET_pair from df_Forman
df_Forman_ET_pair = df_Forman.select(col("Ver"), col("VT_filtra"), col("Forman.ET_pair").alias("ET_pair"))
df_Forman_ET_pair.printSchema()
# df_Forman_ET_pair.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- ET_pair: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ET_pair_E: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- ET_pair_T: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [41]:
# retrieve co-boundary triangles (ET relation)
def get_co_tri(VT, ET_pair):
    if len(ET_pair) > 0:
        co_tri_pair = []
        num_ET_pair = len(ET_pair) # obtain the number of ET_pairs
        def Co_tri(e,t,vt):
            # e is sorted
            for t_temp in vt:
                # e0 = sorted([vt[0], vt[1]], reverse=True) # sort the edge in descending order
                e0 = [t_temp[0], t_temp[1]] # e0, e1, and e2 are in descending order by default
                e1 = [t_temp[0], t_temp[2]]
                e2 = [t_temp[1], t_temp[2]]
                t = [int(t[0]), int(t[1]), int(t[2])]
                t_temp = [int(t_temp[0]), int(t_temp[1]), int(t_temp[2])]
                # if e==e0 or e==e1 or e==e2 and t != t_temp:
                if e==e0 or e==e1 or e==e2:
                    if t != t_temp:
                        return t_temp # t_temp is a coboundary triangle of t in vt
                
        for i in range(num_ET_pair):
            ET_pair_e = ET_pair[i]['ET_pair_E'] # the i-th ET_pair
            ET_pair_t = ET_pair[i]['ET_pair_T']
            ET_pair_co_t = Co_tri(ET_pair_e, ET_pair_t, VT)
            if ET_pair_co_t is None: # len(ET_pair_co_t) is empty, boundary triangles
                ET_pair_co_t = [-1, -1, -1]
            co_tri_pair.append([ET_pair_co_t, ET_pair_t])
            
        return co_tri_pair
    
# the whole StructType
get_co_tri_schema = StructType([
    StructField("ET_pair_co_t", ArrayType(IntegerType()), True),
    StructField("ET_pair_t", ArrayType(IntegerType()), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_co_tri_udf = udf(get_co_tri, ArrayType(get_co_tri_schema))

df_Forman_V2_edge = df_Forman_ET_pair.withColumn("ET_pair_inTri", get_co_tri_udf(df_Forman_ET_pair.VT_filtra, df_Forman_ET_pair.ET_pair))
df_Forman_V2_edge = df_Forman_V2_edge.select(col("Ver"), col("ET_pair_inTri"))
df_Forman_V2_edge.printSchema()
# df_Forman_V2_edge.show()

root
 |-- Ver: integer (nullable = true)
 |-- ET_pair_inTri: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ET_pair_co_t: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- ET_pair_t: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



##### construct Graph_ET reversely, set src nodes as dst nodes, and dst nodes as src nodes

In [42]:
# explode df_Forman_V2_edge
df_Forman_V2_edge_split = df_Forman_V2_edge.select(explode(df_Forman_V2_edge.ET_pair_inTri).alias("ET_pair_src_dst"))

df_Forman_V2_edge_split = df_Forman_V2_edge_split.select(col("ET_pair_src_dst.ET_pair_co_t").alias("arc_dst"), col("ET_pair_src_dst.ET_pair_t").alias("arc_src")).na.drop()
df_Forman_V2_edge_split.printSchema()
# df_Forman_V2_edge_split.show()

root
 |-- arc_dst: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- arc_src: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [43]:
# join two DataFrames by df_Forman_V2_edge_split.arc_src == df_Forman_V2_edge_split.arc_des
# https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
df_Forman_V2_edge_split_src = df_Forman_V2_edge_split.join(df_graph_V2_node, df_Forman_V2_edge_split.arc_src == df_graph_V2_node.tri, "left")
df_Forman_V2_edge_split_src = df_Forman_V2_edge_split_src.select(col("arc_dst"), col("tri_order").alias("src"))
df_Forman_V2_edge_split_src.printSchema()
#df_Forman_V2_edge_split_src.show()

root
 |-- arc_dst: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- src: integer (nullable = true)



In [44]:
# join two DataFrames by df_Forman_V2_edge_split.arc_src == df_Forman_V2_edge_split.arc_des
# https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
df_Forman_V2_edge_split_dst = df_Forman_V2_edge_split_src.join(df_graph_V2_node, df_Forman_V2_edge_split_src.arc_dst == df_graph_V2_node.tri, "left")
df_Forman_V2_edge_split_dst = df_Forman_V2_edge_split_dst.select(col("src"), col("tri_order").alias("dst"))
# df_Forman_V2_edge_split_dst.printSchema()

##### arcs of Graph_ET

In [45]:
df_graph_V2_edge_final = df_Forman_V2_edge_split_dst
df_graph_V2_edge_final.printSchema()
# df_graph_V2_edge_final.show()

root
 |-- src: integer (nullable = true)
 |-- dst: integer (nullable = true)



In [46]:
# df_graph_V2_arc = df_graph_V2_arc.repartition(20)
# print("number of partitions for df_graph_V2_arc:", df_graph_V2_arc.rdd.getNumPartitions())

# save Graph_VE arcs to Parquet
t_save_arc_0 = time.time()
file_df_graph_V2_arc = 'SigSpatial_data' + '/' + tin_filename + '_ET_arc' + '_' + '03132024' + '.parquet'
# df_graph_V2_edge_final.write.parquet(file_df_graph_V2_arc) 

t_save_arc_1 = time.time()
print("Time cost to save Graph_ET arcs in Parquet:", t_save_arc_1 - t_save_arc_0)

Time cost to save Graph_ET arcs in Parquet: 6.890296936035156e-05


##### OR get nodes and arcs from previously saved results

In [10]:
# read nodes of V2
schema_df_graph_V2_node_final = StructType([ \
    StructField("id",IntegerType(),True)
  ])
file_df_graph_V2_node_final_hdfs = 'SigSpatial_data' + '/' + tin_filename + '_graph_V2_node_final.parquet'
df_graph_V2_node_final = spark.read.format("parquet") \
      .option("header", False) \
      .schema(schema_df_graph_V2_node_final)\
      .load(file_df_graph_V2_node_final_hdfs)
print("Number of partitions for df_graph_V2_node_final:", df_graph_V2_node_final.rdd.getNumPartitions())
df_graph_V2_node_final.printSchema()

# read edges of V2
schema_df_graph_V2_edge_final = StructType([ \
    StructField("src",IntegerType(),True), \
    StructField("dst",IntegerType(),True)
  ])
file_df_graph_V2_edge_final_hdfs = 'SigSpatial_data' + '/' + tin_filename + '_graph_V2_edge_final.parquet'
df_graph_V2_edge_final = spark.read.format("parquet") \
      .option("header", False) \
      .schema(schema_df_graph_V2_edge_final)\
      .load(file_df_graph_V2_edge_final_hdfs)
print("Number of partitions for df_graph_V2_edge_final:", df_graph_V2_edge_final.rdd.getNumPartitions())
df_graph_V2_edge_final.printSchema()

Number of partitions for df_graph_V2_node_final: 121
root
 |-- id: integer (nullable = true)

Number of partitions for df_graph_V2_edge_final: 200
root
 |-- src: integer (nullable = true)
 |-- dst: integer (nullable = true)



## Construct a Graph_ET

In [11]:
graph_ET = GraphFrame(df_graph_V2_node_final, df_graph_V2_edge_final)

print("type of graph_V2:", type(graph_ET))

type of graph_V2: <class 'graphframes.graphframe.GraphFrame'>


In [12]:
# set a checkpoint directory to improve performance
# Checkpointing regularly helps recover from failures, clean shuffle files, shorten the
# lineage of the computation graph, and reduce the complexity of plan optimization.

spark.sparkContext.setCheckpointDir('checkpoints')

# Compute descending 2-manifolds (influence regions of maxima), which are the connected components of Graph_ET

In [13]:
t_con_0 = time.time()
# result_con consists of two columns, vertex id, component
result_con = graph_ET.connectedComponents()
result_con.printSchema()

t_con_1 = time.time()
t_con = t_con_1 - t_con_0
print("time cost of connected components:",t_con)

root
 |-- id: integer (nullable = true)
 |-- component: long (nullable = true)

time cost of connected components: 227.86275434494019


##### 1) left join result_con with saddles tri to identify the component that each saddles tri belongs to

In [51]:
# join two DataFrames to obatin VE relation
# https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
df_crit_E_VT = df_crit_E.join(df_VT, df_crit_E.Ver == df_VT.Ver, "left")
df_crit_E_VT = df_crit_E_VT.select(df_crit_E.Ver, "Saddle_edge", "VT_filtra")
df_crit_E_VT.printSchema()
# df_crit_E_VT.show()

root
 |-- Ver: integer (nullable = true)
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [52]:
# retrieve co-boundary triangles (ET relation)
def get_saddle_edge_tri(Saddle_edge, VT):
    if len(VT) > 0:
        e = Saddle_edge
        tri_12 = []
        num_VT = len(VT) # obtain the number of ET_pairs
        for t_temp in VT:
            e0 = [t_temp[0], t_temp[1]] # e0, e1, and e2 are in descending order by default
            e1 = [t_temp[0], t_temp[2]]
            e2 = [t_temp[1], t_temp[2]]
            if e==e0 or e==e1 or e==e2:
                tri_12.append(t_temp)
            # if len(tri_12) == 2: # Maybe this triangle is at the boundary, so it has only one corresponding triangle in VT
                # break
        '''
        if len(tri_12) == 1:
            t_special = [-1,-1,-1]
            tri_12.append(t_special)
        '''
            
        return tri_12
    
# the whole StructType
get_saddle_edge_tri_schema = StructType([
    StructField("Saddle_edge_t1", ArrayType(IntegerType()), True),
    StructField("Saddle_edge_t2", ArrayType(IntegerType()), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_saddle_edge_tri_udf = udf(get_saddle_edge_tri, ArrayType(ArrayType(IntegerType())))

df_crit_E_tri = df_crit_E_VT.withColumn("saddle_edge_Tri", get_saddle_edge_tri_udf(df_crit_E_VT.Saddle_edge, df_crit_E_VT.VT_filtra))
df_crit_E_tri = df_crit_E_tri.select(col("saddle_edge_Tri"))
df_crit_E_tri.printSchema()
# df_crit_E_tri.show()

root
 |-- saddle_edge_Tri: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [53]:
# obtain the saddle tri
df_crit_E2tri = df_crit_E_tri.select(explode("saddle_edge_Tri").alias("saddle_edge_Tri_split"))
# some nodes may belong to multiple saddles, we need to remove them
df_crit_E2tri = df_crit_E2tri.distinct()
df_crit_E2tri.printSchema()

root
 |-- saddle_edge_Tri_split: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [54]:
# left join df_crit_E2tri with df_graph_V2_node to obtain corresponding id of tri
df_crit_E2tri = df_crit_E2tri.join(df_graph_V2_node, df_crit_E2tri.saddle_edge_Tri_split == df_graph_V2_node.tri, "left")
df_crit_E2tri = df_crit_E2tri.select(col('tri_order').alias('saddle_edge_Tri_split_id'))
df_crit_E2tri.printSchema()

root
 |-- saddle_edge_Tri_split_id: integer (nullable = true)



In [17]:
# OR get df_crit_E2pts from previously saved results

schema_df_crit_E2tri = StructType([ \
    StructField("saddle_edge_Tri_split_id",IntegerType(),True)
  ])

file_df_crit_E2tri_hdfs = 'hdfs_data' + '/' + tin_filename + '_crit_E2tri.parquet'
#file_df_crit_E2tri_hdfs = 'SigSpatial_data' + '/' + tin_filename + '_crit_E2tri.parquet'

df_crit_E2tri = spark.read.format("parquet") \
      .option("header", False) \
      .schema(schema_df_crit_E2tri)\
      .load(file_df_crit_E2tri_hdfs)

df_crit_E2tri_file.printSchema()

root
 |-- saddle_edge_Tri_split_id: integer (nullable = true)



In [18]:
result_con_ET = result_con
# identify the saddle tri that each component have
result_con_saddle_ET = result_con_ET.join(df_crit_E2tri,result_con_ET.id==df_crit_E2tri.saddle_edge_Tri_split_id, "inner")
result_con_saddle_ET = result_con_saddle_ET.withColumn("att", when(result_con_saddle_ET.saddle_edge_Tri_split_id.isNull() == False, result_con_saddle_ET.saddle_edge_Tri_split_id).otherwise(-1)).drop('saddle_edge_Tri_split_id')

print("number of partitions for result_con_saddle:", result_con_saddle_ET.rdd.getNumPartitions())
result_con_saddle_ET.printSchema()
# result_con_saddle.show(30)

number of partitions for result_con_saddle: 200
root
 |-- id: integer (nullable = true)
 |-- component: long (nullable = true)
 |-- att: integer (nullable = true)



In [19]:
# obtain all saddle tris for each connected component
unique_SdlTris_per_con_ET = result_con_saddle_ET.groupBy('component').agg(collect_list('id').alias('multi_SdlTris'))
unique_SdlTris_per_con_ET.printSchema()

root
 |-- component: long (nullable = true)
 |-- multi_SdlTris: array (nullable = true)
 |    |-- element: integer (containsNull = true)



##### 2) left join df_graph_V2_edge_final with result_con to identify the connected component that each arc belongs to

In [20]:
# left join df_graph_V1_arc with result_con to identify which component that each arc belongs to
df_G_V2_arc_component = df_graph_V2_edge_final.join(result_con_ET,df_graph_V2_edge_final.src==result_con_ET.id, "left").drop('id')
df_G_V2_arc_component.printSchema()

root
 |-- src: integer (nullable = true)
 |-- dst: integer (nullable = true)
 |-- component: long (nullable = true)



##### df_graph_V2_arc_component is the arcs of Graph_ET

In [21]:
Num_subgraphs = 20
t_find_minmax_0 = time.time()

# Find the minimum and maximum values of "component" in result_con
sub_graph_tmp_min = result_con.agg({"component": "min"}).collect()[0][0]
sub_graph_tmp_max = result_con.agg({"component": "max"}).collect()[0][0]

sub_graph_list = []
for i in range(Num_subgraphs+1):
    sub_graph_tmp = int(i * (sub_graph_tmp_max - sub_graph_tmp_min)/(Num_subgraphs) + sub_graph_tmp_min)
    sub_graph_list.append(sub_graph_tmp)
    
'''

# sort and collect result_con_unique
result_con_unique = result_con.groupBy('component').count()
# result_con_unique.printSchema()
# result_con_unique.show()

result_con_unique = result_con_unique.sort(col('component'))
result_con_unique_col = result_con_unique.collect()

sub_graph_list = []

for i in range(Num_subgraphs):
    tmp = int(i / (Num_subgraphs-1) * (len(result_con_unique_col) -1))
    sub_graph_tmp = result_con_unique_col[tmp]['component']
    sub_graph_list.append(sub_graph_tmp)
'''    

t_find_minmax_1 = time.time()
t_find_minmax = t_find_minmax_1 - t_find_minmax_0
print("time cost to find sub_graph_list:", t_find_minmax)

time cost to find sub_graph_list: 6.009696960449219


In [22]:
# Create empty DataFrame with a schema
schema_special = StructType([
    StructField('id', LongType(),True),
    StructField("component", LongType(),True),
    StructField("att", LongType(),True),   
    StructField("label", LongType(),True)   
])

df_empty = spark.createDataFrame([], schema_special)

In [23]:
# construct subgraphs and union the LPA results
t_LPA_total_0 = time.time()

sub_time = [] # to store time cost for each subgraph
for i in range(len(sub_graph_list) - 1):
    t_LPA_sub_0 = time.time()
    sub_graph_tmp_0 = sub_graph_list[i]
    sub_graph_tmp_1 = sub_graph_list[i+1]
    
    if i == len(sub_graph_list) - 1: # the last subgraphs should contain sub_graph_tmp_max
        sub_graph_node = result_con_saddle_ET.filter((result_con_saddle_ET.component >= sub_graph_tmp_0) & (result_con_saddle_ET.component <= sub_graph_tmp_1))
        sub_graph_arc = df_G_V2_arc_component.filter((df_G_V2_arc_component.component >= sub_graph_tmp_0) & (df_G_V2_arc_component.component <= sub_graph_tmp_1)).drop('id')
    else:
        sub_graph_node = result_con_saddle_ET.filter((result_con_saddle_ET.component >= sub_graph_tmp_0) & (result_con_saddle_ET.component < sub_graph_tmp_1))
        # sub_graph_node.printSchema() # 3 columns: id, component, att
        sub_graph_arc = df_G_V2_arc_component.filter((df_G_V2_arc_component.component >= sub_graph_tmp_0) & (df_G_V2_arc_component.component < sub_graph_tmp_1)).drop('id')
        # sub_graph_arc.printSchema() # 3 columns: src, dst, component
    
    sub_graph = GraphFrame(sub_graph_node, sub_graph_arc)
    
    sub_graph_LPA = sub_graph.pregel \
         .setMaxIter(10) \
         .withVertexColumn("label", col("att"), \
             Pregel.msg()) \
         .sendMsgToDst(when(Pregel.dst("label")>0, Pregel.dst("label")).otherwise(Pregel.src("label"))) \
         .aggMsgs(F.max(Pregel.msg())) \
         .run()
    t_LPA_sub_1 = time.time()
    df_empty = df_empty.union(sub_graph_LPA)
    print("Time cost for LPA to subgraph %ld: %f" % (sub_graph_tmp_0, t_LPA_sub_1 - t_LPA_sub_0))
    sub_time.append(t_LPA_sub_1 - t_LPA_sub_0)
    
t_LPA_total_1 = time.time()
print("*******************************************************")
print("Total time cost for LPA:", t_LPA_total_1 - t_LPA_total_0)

Time cost for LPA to subgraph -1: 112.242347
Time cost for LPA to subgraph 4898863: 87.272774
Time cost for LPA to subgraph 9797727: 88.380053
Time cost for LPA to subgraph 14696591: 84.740761
Time cost for LPA to subgraph 19595455: 84.608591
Time cost for LPA to subgraph 24494320: 79.805896
Time cost for LPA to subgraph 29393184: 79.376734
Time cost for LPA to subgraph 34292048: 80.889919
Time cost for LPA to subgraph 39190912: 74.401150
Time cost for LPA to subgraph 44089776: 74.403847
Time cost for LPA to subgraph 48988641: 76.381688
Time cost for LPA to subgraph 53887505: 75.135764
Time cost for LPA to subgraph 58786369: 84.505844
Time cost for LPA to subgraph 63685233: 74.865218
Time cost for LPA to subgraph 68584097: 77.825959
Time cost for LPA to subgraph 73482962: 76.659230
Time cost for LPA to subgraph 78381826: 82.205308
Time cost for LPA to subgraph 83280690: 77.444861
Time cost for LPA to subgraph 88179554: 77.599617
Time cost for LPA to subgraph 93078418: 76.883111
*******

In [24]:
print("time cost of connected components:",t_con)

print("time cost to find sub_graph_list:", t_find_minmax)

print("Total time cost for LPA:", t_LPA_total_1 - t_LPA_total_0)

time cost of connected components: 227.86275434494019
time cost to find sub_graph_list: 6.009696960449219
Total time cost for LPA: 1625.8993074893951
